## Time-Series Regression Analysis for Long-Short and Beta-Neutral Strategies
#### Purpose:
This section of the code is designed to replicate the time-series regression analysis presented in the paper "Do Common Factors Really Explain the Cross-Section of Stock Returns?" (3.5 Time-Series Regressions). The code implements regression models using popular factor models, such as CAPM, Fama-French Five Factors, q5, and the mispricing model. It calculates factor loadings (betas), alphas, and evaluates model fit to understand the explanatory power of these factors on the Long-Short and Beta-Neutral portfolios.

#### Author:
- **Author:** Wenxin (Wency) Jiang  
- **Date Created:** December 7, 2024  

#### Key Notes:
- The analysis includes two types of portfolios: Long-Short (unhedged) and Beta-Neutral (hedged).
- Regression outputs will highlight significant factor exposures and alphas.
- The results aim to replicate the findings and validate the robustness of the paper's conclusions.

--- Data Sources ---
- **Fama-French Five-Factor Data - Fama and French**  
    - **Extension:** Alejandro and Nikolai used Fama-French Five-Factor Data from Fama and French (2015) from 1974 to 2021 for their time series regression analysis. With additional data now available, we extended the data set to include information up to 2023-12-31.  
    - **Reference:** Fama/French 5 Research Factors, Kenneth R. French, [Kenneth R. French's Website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/index.html), December 7, 2024.  

- **q5 model - Hou et al.**  
    - **Reference:** q, Hou, Kewei, Chen Xue, and Lu Zhang, [Global-q Factors](http://global-q.org/factors.html), December 7, 2024.  

- **Mispricing Factors model - Stambaugh and Yuan**  
    - **Reduction:** Alejandro and Nikolai utilized data from 1974-2021. However, due to data unavailability, we will only use 1974-2016 data for time series analysis.  
    - **Reference:** Robert F. Stambaugh, [WRDS Data](https://wrds-www.wharton.upenn.edu/pages/get-data/financial-ratios-suite-wrds/financial-ratios/financial-ratios-firm-level-by-wrds-beta/), December 7, 2024.  
---


In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import date, datetime

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas_datareader.data as web # to download Fama-French factors, q5, mispricing
import statsmodels.api as sm # to run regression

SEED = 42

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# import warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# import data
stock_data = pd.read_csv('./data/stock_data.csv') # prepared in 3.1

## Table 10 

In [ ]:
# Prepare data for Fama-French 5-Factor Model and Momentum Factor
def get_factor_data():
    # 下载 Fama-French 5因子数据和动量因子
    ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', 
                        start='1974-01', end='2023-12')[0] / 100  # 转换为小数
    mom = web.DataReader('F-F_Momentum_Factor', 'famafrench', 
                        start='1974-01', end='2023-12')[0] / 100
    
    # 合并因子数据
    # 使用 FF5 中除 RF 外的所有列
    factors = ff5.drop('RF', axis=1)
    # 重命名动量因子列（注意原列名中有额外的空格）
    mom = mom.rename(columns={'Mom   ': 'Mom'})
    # 合并数据
    factors = pd.concat([factors, mom['Mom']], axis=1)
    
    return factors

factors = get_factor_data()

In [ ]:
# 2. create beta and portfolio returns
def calculate_beta(returns, market_returns, window=12):
    """计算滚动beta"""
    covariance = returns.rolling(window=window).cov(market_returns)
    market_variance = market_returns.rolling(window=window).var()
    return covariance / market_variance

def construct_monthly_portfolios(stock_df):
    # 转换为月度数据
    monthly_df = stock_df.groupby(['PERMNO', 'YEAR', 'MONTH']).agg({
        'RET': lambda x: (1 + x).prod() - 1,
        'MKT': lambda x: (1 + x).prod() - 1,
        'CAP': 'last'
    }).reset_index()
    
    # 计算beta
    monthly_df['beta'] = monthly_df.groupby('PERMNO').apply(
        lambda x: calculate_beta(x['RET'], x['MKT'])
    ).reset_index(0, drop=True)
    
    # 计算动量
    monthly_df['momentum'] = monthly_df.groupby('PERMNO')['RET'].rolling(
        window=12, min_periods=12
    ).apply(lambda x: (1 + x).prod() - 1).reset_index(0, drop=True)
    
    portfolios = []
    for (year, month), month_data in monthly_df.groupby(['YEAR', 'MONTH']):
        # 移除缺失值
        month_data = month_data.dropna(subset=['beta', 'momentum', 'RET'])
        
        # 检查是否有足够的数据进行分组
        if len(month_data) >= 10:  # 确保有足够的数据进行十分位分组
            try:
                # 按动量分组
                month_data['decile'] = pd.qcut(month_data['momentum'], 
                                             q=10, 
                                             labels=False, 
                                             duplicates='drop')
                
                # 确保分组成功后再计算收益率
                if not month_data['decile'].isna().all():
                    # 计算多空组合收益率
                    long_short = month_data[month_data['decile'] == 9]['RET'].mean() - \
                                month_data[month_data['decile'] == 0]['RET'].mean()
                    
                    # 计算beta中性组合收益率
                    high_momentum = month_data[month_data['decile'] == 9]
                    low_momentum = month_data[month_data['decile'] == 0]
                    
                    beta_neutral = (high_momentum['RET'] / high_momentum['beta']).mean() - \
                                 (low_momentum['RET'] / low_momentum['beta']).mean()
                    
                    portfolios.append({
                        'YEAR': year,
                        'MONTH': month,
                        'Long-Short': long_short,
                        'Beta-Neutral': beta_neutral
                    })
            except Exception as e:
                print(f"跳过 {year}-{month} 的计算: {str(e)}")
                continue
    
    return pd.DataFrame(portfolios)

portfolio_returns = construct_monthly_portfolios(stock_df)

In [ ]:
# 调整 portfolio_returns 的格式
def prepare_portfolio_returns(portfolio_returns):
    """
    准备投资组合收益率数据，将年月转换为日期索引
    """
    # 创建日期索引
    portfolio_returns['Date'] = pd.to_datetime(
        portfolio_returns['YEAR'].astype(str) + '-' + 
        portfolio_returns['MONTH'].astype(str) + '-01'
    )
    
    # 设置日期索引并只保留收益率列
    portfolio_returns_formatted = portfolio_returns[['Date', 'Long-Short', 'Beta-Neutral']].copy()
    portfolio_returns_formatted = portfolio_returns_formatted.set_index('Date')
    
    return portfolio_returns_formatted

portfolio_returns = prepare_portfolio_returns(portfolio_returns)
print("Portfolio returns structure:")
print(portfolio_returns.head())
print("\nPortfolio returns columns:", portfolio_returns.columns)

In [ ]:
def run_regressions(portfolio_returns, factors):
    """
    Time Series Regression of the Long-Short and the Hedged Portfolio
    against the CAPM and Fama-French Five Factors plus Momentum
    
    portfolio_returns: DataFrame, preprocessed portfolio returns
    factors: DataFrame, Fama-French Five Factors plus Momentum
    """
    # 打印数据结构，帮助调试
    print("Portfolio returns columns:", portfolio_returns.columns)
    print("Factors columns:", factors.columns)
    
    # 准备回归模型
    models = {
        'CAPM': ['Mkt-RF'],
        'FF5_Mom': ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom']
    }
    
    results = {}
    
    # 确保因子数据的索引是 datetime
    if isinstance(factors.index, pd.PeriodIndex):
        factors.index = factors.index.to_timestamp()
    
    # 找到共同的日期
    common_dates = portfolio_returns.index.intersection(factors.index)
    
    # 对齐数据
    portfolio_returns = portfolio_returns.loc[common_dates]
    factors = factors.loc[common_dates]
    
    for model_name, model_factors in models.items():
        # 准备数据
        X = sm.add_constant(factors[model_factors])
        
        # 对每个组合运行回归
        y_long_short = portfolio_returns['Long-Short']
        model_long_short = sm.OLS(y_long_short, X).fit(cov_type='HAC', cov_kwds={'maxlags': 6})
        results[f'Long-Short_{model_name}'] = model_long_short
        
        y_beta_neutral = portfolio_returns['Beta-Neutral']
        model_beta_neutral = sm.OLS(y_beta_neutral, X).fit(cov_type='HAC', cov_kwds={'maxlags': 6})
        results[f'Beta-Neutral_{model_name}'] = model_beta_neutral
    
    return results

regression_results = run_regressions(portfolio_returns, factors)

In [ ]:
def replicate_Table_10(regression_results):
    """
    创建回归结果表格
    """
    def format_coef(coef, tstat):
        stars = ''
        if abs(tstat) > 3.291:  # p < 0.001
            stars = '***'
        elif abs(tstat) > 2.576:  # p < 0.01
            stars = '**'
        elif abs(tstat) > 1.96:  # p < 0.05
            stars = '*'
        return f"{coef:.2f}{stars}", f"({tstat:.2f})"

    # 创建图表
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')

    # 准备数据
    models = {
        'Long-Short_CAPM': regression_results['Long-Short_CAPM'],
        'Beta-Neutral_CAPM': regression_results['Beta-Neutral_CAPM'],
        'Long-Short_FF5_Mom': regression_results['Long-Short_FF5_Mom'],
        'Beta-Neutral_FF5_Mom': regression_results['Beta-Neutral_FF5_Mom']
    }

    # 创建表格内容
    rows = ['Intercept', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'Adj. R²', 'Num. obs.']
    data = []
    
    for row in rows:
        row_data = []
        for model in models.values():
            if row == 'Adj. R²':
                row_data.append(f"{model.rsquared_adj:.2f}")
            elif row == 'Num. obs.':
                row_data.append(f"{int(model.nobs)}")
            else:
                try:
                    param_name = 'const' if row == 'Intercept' else row
                    if param_name in model.params.index:
                        coef = model.params[param_name]
                        tstat = model.tvalues[param_name]
                        coef_str, tstat_str = format_coef(coef, tstat)
                        row_data.append(f"{coef_str}\n{tstat_str}")
                    else:
                        row_data.append('')
                except Exception as e:
                    print(f"Error processing {row}: {str(e)}")
                    row_data.append('')
        data.append(row_data)

    # 创建表格
    table = ax.table(
        cellText=data,
        rowLabels=rows,
        colLabels=['Long-Short', 'Beta-Neutral', 'Long-Short', 'Beta-Neutral'],
        cellLoc='center',
        loc='center',
        bbox=[0.1, 0, 0.8, 0.9]
    )

    # 设置表格样式
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    # 添加标题
    plt.title('Table 10: Time Series Regression of the Long-Short and the Hedged Portfolio\n'
              'against the CAPM and Fama-French Five Factors plus Momentum', 
              pad=20, fontsize=10)
    
    # 添加公式
    formula = r"$Portfolio_t = \alpha + \sum_{i=1}^{5} \beta_iF_{it} + \epsilon_t$"
    plt.figtext(0.5, 0.95, formula, fontsize=10, ha='center')
    
    # 添加注释
    footnote = ("Notes: This table reports the regression of monthly stock returns (in percent) of the long-\n"
                "short portfolio and beta-neutral portfolio on the CAPM and the Fama-French five-factor\n"
                "model. The t-statistic are shown in parenthesis. The sample period is 1974 to 2023.\n"
                "*p<0.1; **p<0.05; ***p<0.01.")
    plt.figtext(0.1, -0.1, footnote, fontsize=8, ha='left', va='top')

    # 保存图片
    plt.savefig('./replicate_Table_10.png', bbox_inches='tight', dpi=300)
    plt.show()

# 使用函数
try:
    replicate_Table_10(regression_results)
    print("表格已保存为 'replicate_Table_10.png'")
except Exception as e:
    print(f"创建表格时发生错误: {str(e)}")
    import traceback
    traceback.print_exc()

## Table 11

In [ ]:
# prepare data for q5 model and mispricing model
def get_q5_factors():
    """
    q5 model of Hou et al. (2021)
    http://global-q.org/factors.html
    "R_F" stands for the one-month Treasury bill rates, 
    "R_MKT" the market excess returns, 
    "R_ME" the size factor returns, 
    "R_IA" the investment factor returns, 
    "R_ROE" the return on equity factor returns,
    "R_EG" the expected growth factor returns.
    """
    # 读取数据
    q5_factors = pd.read_csv('./data/q5_factors_monthly_2023.csv')
    
    # 创建日期列
    q5_factors['date'] = pd.to_datetime(q5_factors[['year', 'month']].assign(day=1))
    
    # 选择需要的列并重命名
    q5_factors = q5_factors[['date', 'R_MKT', 'R_ME', 'R_IA', 'R_ROE', 'R_EG']]
    q5_factors.columns = ['date', 'Mkt-RF', 'R_ME', 'R_IA', 'R_ROE', 'R_EG']  # 重命名以匹配表格
    
    # 设置日期索引
    q5_factors = q5_factors.set_index('date')
    
    # 筛选1974-2023的数据
    q5_factors = q5_factors[q5_factors.index.year >= 1974]
    
    # 转换为小数
    q5_factors = q5_factors / 100
    
    return q5_factors



def get_mispricing_factors():
    """
    mispricing model of Stambaugh and Yuan (2017)
    https://finance.wharton.upenn.edu/~stambaug/
    """
    # 读取数据
    mispricing_factors = pd.read_csv('./data/M4(1963-2016).csv')
    
    # 创建日期列
    mispricing_factors['date'] = pd.to_datetime(
        mispricing_factors['YYYYMM'].astype(str), format='%Y%m'
    )
    
    # 选择需要的列并重命名
    mispricing_factors = mispricing_factors[['date', 'MKTRF', 'SMB', 'MGMT', 'PERF']]
    mispricing_factors.columns = ['date', 'Mkt-RF', 'SMB', 'MGMT', 'PERF']
    
    # 设置日期索引
    mispricing_factors = mispricing_factors.set_index('date')
    
    # 筛选1974-2016的数据
    mispricing_factors = mispricing_factors[
        (mispricing_factors.index.year >= 1974)
    ]
    
    # 转换为小数
    mispricing_factors = mispricing_factors / 100
    
    return mispricing_factors

def prepare_regression_data(portfolio_returns):
    """
    准备两组回归数据
    """
    # 获取因子数据
    q5_factors = get_q5_factors()
    mispricing_factors = get_mispricing_factors()
    
    # 准备q5模型数据 (1974-2023)
    q5_common_dates = portfolio_returns.index.intersection(q5_factors.index)
    q5_portfolio_returns = portfolio_returns.loc[q5_common_dates]
    q5_factors = q5_factors.loc[q5_common_dates]
    
    # 准备mispricing模型数据 (1974-2016)
    mispricing_common_dates = portfolio_returns.index.intersection(mispricing_factors.index)
    mispricing_portfolio_returns = portfolio_returns.loc[mispricing_common_dates]
    mispricing_factors = mispricing_factors.loc[mispricing_common_dates]
    
    return {
        'q5': {
            'portfolio_returns': q5_portfolio_returns,
            'factors': q5_factors
        },
        'mispricing': {
            'portfolio_returns': mispricing_portfolio_returns,
            'factors': mispricing_factors
        }
    }


regression_data = prepare_regression_data(portfolio_returns)
q5_data = regression_data['q5']
mispricing_data = regression_data['mispricing']

In [ ]:
def run_q5_regression(portfolio_returns, q5_factors):
    X = sm.add_constant(q5_factors)  # 添加常数项
    
    results = {}
    
    for portfolio_type in ['Long-Short', 'Beta-Neutral']:
        y = portfolio_returns[portfolio_type]
        # Use Newey-West std(HAC) regression
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 6})
        results[portfolio_type] = model
    
    return results

def run_mispricing_regression(portfolio_returns, mispricing_factors):
    X = sm.add_constant(mispricing_factors)  # 添加常数项
    
    results = {}

    for portfolio_type in ['Long-Short', 'Beta-Neutral']:
        y = portfolio_returns[portfolio_type]
        # Use Newey-West std(HAC) regression
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 6})
        results[portfolio_type] = model
    
    return results

def create_regression_table(q5_results, mispricing_results):
    def format_coef(coef, tstat):
        stars = ''
        if abs(tstat) > 3.291:  # p < 0.001
            stars = '***'
        elif abs(tstat) > 2.576:  # p < 0.01
            stars = '**'
        elif abs(tstat) > 1.96:  # p < 0.05
            stars = '*'
        return f"{coef:.2f}{stars}", f"({tstat:.2f})"

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')

    rows = ['Intercept', 'Mkt-RF', 'R_ME', 'R_IA', 'R_ROE', 'R_EG', 
            'SMB', 'MGMT', 'PERF', 'Adj. R²', 'Num. obs.']
    
    data = []
    
    for row in rows:
        row_data = []
        # q5
        for model in [q5_results['Long-Short'], q5_results['Beta-Neutral']]:
            if row == 'Adj. R²':
                row_data.append(f"{model.rsquared_adj:.2f}")
            elif row == 'Num. obs.':
                row_data.append(f"{int(model.nobs)}")
            else:
                try:
                    param_name = 'const' if row == 'Intercept' else row
                    if param_name in model.params.index:
                        coef = model.params[param_name]
                        tstat = model.tvalues[param_name]
                        coef_str, tstat_str = format_coef(coef, tstat)
                        row_data.append(f"{coef_str}\n{tstat_str}")
                    else:
                        row_data.append('')
                except:
                    row_data.append('')
        
        # mispricing
        for model in [mispricing_results['Long-Short'], mispricing_results['Beta-Neutral']]:
            if row == 'Adj. R²':
                row_data.append(f"{model.rsquared_adj:.2f}")
            elif row == 'Num. obs.':
                row_data.append(f"{int(model.nobs)}")
            else:
                try:
                    param_name = 'const' if row == 'Intercept' else row
                    if param_name in model.params.index:
                        coef = model.params[param_name]
                        tstat = model.tvalues[param_name]
                        coef_str, tstat_str = format_coef(coef, tstat)
                        row_data.append(f"{coef_str}\n{tstat_str}")
                    else:
                        row_data.append('')
                except:
                    row_data.append('')
        
        data.append(row_data)

    table = ax.table(
        cellText=data,
        rowLabels=rows,
        colLabels=['Long-Short', 'Beta-Neutral', 'Long-Short', 'Beta-Neutral'],
        cellLoc='center',
        loc='center',
        bbox=[0.1, 0, 0.8, 0.9]
    )

    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    # 添加标题
    plt.title('Table 11: Time Series Regression of the Long-Short and the Hedged Portfolio\n'
              'against the q5 and the Mispricing Model', 
              pad=20, fontsize=10)
    
    # 添加公式
    formula = r"$Portfolio_t = \alpha + \sum_{i=1}^{5} \beta_iF_{it} + \epsilon_t$"
    plt.figtext(0.5, 0.95, formula, fontsize=10, ha='center')
    
    # 添加注释
    footnote = ("Notes: This table reports the regression of monthly stock returns (in percent) of the long-\n"
                "short portfolio and beta-neutral portfolio on the q5 factors from Hou et al. (2021) and the\n"
                "mispricing factors from Stambaugh and Yuan (2017). The t-statistics are shown in parentheses.\n"
                "The sample period is 1974 to 2023 for q5 model and 1974 to 2016 for mispricing model.(due to data availability)\n"
                "*p<0.1; **p<0.05; ***p<0.01")
    plt.figtext(0.1, -0.1, footnote, fontsize=8, ha='left', va='top')

    plt.savefig('./table_11.png', bbox_inches='tight', dpi=300)
    plt.show()

regression_data = prepare_regression_data(portfolio_returns)

# q5
q5_results = run_q5_regression(
    regression_data['q5']['portfolio_returns'],
    regression_data['q5']['factors']
)
# mispricing
mispricing_results = run_mispricing_regression(
    regression_data['mispricing']['portfolio_returns'],
    regression_data['mispricing']['factors']
)

# create table
create_regression_table(q5_results, mispricing_results)